In [ ]:
import requests
import pandas as pd
import json
import ast

class readURL:
    data: pd.DataFrame

    def __init__(self, url: str):
        self.url = url
        
    def read(self):
        payload = ""
        headers = {}
        response = requests.request("GET", self.url, headers=headers, data=payload)
        return response
    
    def text(self):
        read = self.read().text
        return read

    def only_dict(self, d):
        return ast.literal_eval(d)

    def list_of_dicts(self, ld):
        return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

    def expand(self, withId:bool = True) -> pd.DataFrame:
        collumns = self.data.columns
        for col in collumns:
            if self.data[col].dtype == object and self.data[col].name != '_id' :
                #if withId:
                newColumn = self.data[col].tolist()
                print(newColumn)
                #newColumn = pd.json_normalize(newColumn).add_prefix(str(self.data[col].name) + '.')
                #else:
                    #newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).drop(columns=['_id']).add_prefix(str(data[x].name) + '.')
                #data = data.drop(columns=data[x].name)
                #data = data.join([newColumn])
        return self.data

    def dataFrame(self, withId:bool = True, columnExpand:bool = True) -> pd.DataFrame:
        self.data = pd.read_json(self.text())
        if not withId:
            self.data = self.data.drop(columns=['_id'])
        if columnExpand:
            self.data = self.expand(withId=withId)
        return self.data
  

In [ ]:
from pymongo import MongoClient
import pymongo
from django.http.response import JsonResponse
from bson.json_util import dumps, loads

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['CeDRI_UGV']['ActionsApplication_queue'].aggregate([
    {
        '$sort': {
            'dateTime': -1
        }
    }, {
        '$limit': 1
    }
])

list(result)[0]

In [ ]:
battery = readURL("http://127.0.0.1:8000/battery/battery")
df = battery.dataFrame(withId=False)
df

In [ ]:

import requests
import pandas as pd
import json
import ast

class readJSON:
    def __init__(self, json, pandas: pd.DataFrame):
        self.json = json
        if pandas == None:
            self.pandas = pd.read_json(self.json)

    def only_dict(self, d):
        return ast.literal_eval(d)

    def list_of_dicts(self, ld):
        return dict([(list(d.values())[1], list(d.values())[0]) for d in ast.literal_eval(ld)])

    def expand(self, withId:bool = True) -> pd.DataFrame:
        data = self.pandas
        for x in data.columns:
            if data[x].dtypes == object and data[x].name != '_id' :
                if withId:
                    newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).add_prefix(str(data[x].name) + '.')
                else:
                    newColumn = pd.json_normalize(data[x].apply(ast.literal_eval).tolist()).drop(columns=['_id']).add_prefix(str(data[x].name) + '.')
                data = data.drop(columns=data[x].name)
                data = data.join([newColumn])
        return data

    def dataFrame(self, withId:bool = True, columnExpand:bool = True) -> pd.DataFrame:
        data = self.pandas
        if not withId:
            self.pandas = self.pandas.drop(columns=['_id'])
        if columnExpand:
            self.pandas = self.expand(withId=withId)
        self.json = data.to_json()
        return data

In [ ]:
json_ = readJSON(battery.json())
json_.dataFrame(withId=False)

In [ ]:
import requests
import json
import bson

url = "http://127.0.0.1:8000/actions/queue"
url1 = "http://127.0.0.1:8000/actions/actions/1"

_json = {
  "QueueNumber": "2",
  "Action": bson.ObjectId(oid='63946e374bb315a0e907d6dc')
}
#payload = json.dumps(_json)
headers = {
  'Content-Type': 'application/json'
}

#response = requests.request("Ge", url, headers=headers, data=payload)
response = requests.request("GET", url1)

response.text




In [ ]:
import datetime
_bson = {
  "QueueNumber": "2",
  "Action": bson.ObjectId(oid='63946e374bb315a0e907d6dc')
}
_json

In [ ]:
from pymongo import MongoClient

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['CeDRI_UGV']['ActionsApplication_queue'].aggregate([
    {
        '$limit': 1
    }
])
_bson = {
  "QueueNumber": "5",
  "Action_id": bson.ObjectId(oid='63946e374bb315a0e907d6dd')
}
try:
    result = client['CeDRI_UGV']['ActionsApplication_queue'].insert_one(_bson)
    result
except:
    print('Fail')


In [ ]:
_bson = {
  "QueueNumber": "5",
  "Action_id": bson.ObjectId(oid='63946e374bb315a0e907d6dd')
}
_bson

In [ ]:
import requests
import json
from random import randint, randbytes, random

url = "http://127.0.0.1:8000/position/position"

for x in range(0,1000):
    payload = json.dumps({
        "Odometry": {
            "Left": random(),
            "Right": random()
        },
        "Fiducial": {
            "Mark": chr(randint(65,68))
        },
        "Gyroscope": {
            "Angle": random()
        },
        "GlobalPosition": {
            "X": random()*100,
            "Y": random()*100
        }
    })
    headers = {
    'Content-Type': 'application/json'
    }

    response = requests.request("POST", url, headers=headers, data=payload)
    print(response.text)

    



In [ ]:
import requests
import json

url = "http://127.0.0.1:8000/route/route"

payload = json.dumps({
  "Estimated": {
    "Distance": "0.1",
    "Consumption": "0.5"
  },
  "Real": {
    "Distance": "0.9",
    "Consumption": "0.6"
  },
  
})
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


In [10]:
from pymongo import MongoClient
import bson
from random import random

# Requires the PyMongo package.
# https://api.mongodb.com/python/current

client = MongoClient('mongodb://localhost:27017/')
result = client['CeDRI_UGV']['RouteApplication_route'].aggregate([
    {
        '$limit': 1
    }
])

_bson = {
    "Estimated": {
        "Distance": random()*10,
        "Consumption": random()*10
    },
    "Real": {
        "Distance": random()*10,
        "Consumption": random()*10
    },
    "StartPoint": bson.ObjectId(oid='6395bbc3a904844e4fd44f96'),
    "EndPoint": bson.ObjectId(oid='6395bbc4a904844e4fd44f99')
}
try:
    result = client['CeDRI_UGV']['RouteApplication_route'].insert_one(_bson)
    result
except:
    print('Fail')

Fail
